# A Two-Hours Seminar about [InterCom](https://github.com/Tecnologias-multimedia/intercom)

## 1. What is InterCom
InterCom is a real-time intercommunicator (currently only audio). The sequence of frames (two stereo samples) is splitted into chunks and each one is transmitted in an [UDP](https://en.wikipedia.org/wiki/User_Datagram_Protocol) packet.

## 2. Buffering
To hide the network [jitter](https://en.wikipedia.org/wiki/Jitter).

![Buffering](https://tecnologias-multimedia.github.io/study_guide/buffering/graphics/timelines.svg)

In [ ]:
!python ../minimal.py --show_stats

In [ ]:
!python ../buffer.py --show_stats

## 3. Compression using [DEFLATE](https://en.wikipedia.org/wiki/DEFLATE)
The chunks are compressed using [LZSS](https://en.wikipedia.org/wiki/Lempel-Ziv-Storer-Szymanski) (that is based on [LZ77](https://github.com/vicente-gonzalez-ruiz/LZ77)) and [Huffman Coding](https://en.wikipedia.org/wiki/Huffman_coding) (see also [this](https://vicente-gonzalez-ruiz.github.io/Huffman_coding/)). This text compression algorithm is used because:
1. Works well when repeated strings are found at the input.
2. Is included with [The Standard Python Library](https://docs.python.org/3/library/) ([zlib](https://docs.python.org/3/library/zlib.html)).

## 4. Bit-rate control through [quantization](https://github.com/vicente-gonzalez-ruiz/quantization/blob/master/digital_quantization.ipynb)

Quantization removes the less relevant information (mainly [electronic noise](https://en.wikipedia.org/wiki/Noise_(electronics)) ...) to increase the [compression ratio](https://en.wikipedia.org/wiki/Compression_ratio). A dead-zone quantizer is applied to the transform coefficients (generated after the spatial and the temporal decorrelation).

Controlling the quantization step, it's possible to control the transmission bit-rate.

## 5. Spatial (inter-channel) decorrelation
The samples of a (stero) frame tend to have similar amplitudes. We apply [Mid/Side coding](https://tecnologias-multimedia.github.io/study_guide/spatial_decorrelation/).

## 6. Temporal (intra-channel) decorrelation
The samples of a channel exhibit temporal redundancy. We use a [Discrete Wavelet Transform (DWT)](https://tecnologias-multimedia.github.io/study_guide/temporal_decorrelation/) to exploit it.

## 7. Work to do
1. Currently, quantization minimizes the error in L$_2$ (the Mean Square Error). Perceptual quantization can increase the compression ratio without increasing the perceived distortion.
2. Transmit video.